In [1]:
import os
import time
import pandas as pd
from web3 import HTTPProvider, Web3
from pachira import *

In [2]:
chain = Net.POLYGON
platform = Platform.SUSHI
contract = JSONContract.UniswapV2Pair

json_rpc_url = os.environ.get(RPC.get_key(chain), RPC.get_rpc(chain))
web3 = Web3(HTTPProvider(json_rpc_url))
web3.middleware_onion.clear()

In [3]:
# Get contracts
abi_path = platform + '/' + contract + '.json'
Pair = ABILoading().get_contract(web3, abi_path)
swap_filt = Filter.create_filter(address=None, event_types=[Pair.events.Swap])  # Listen events from any smart contract

In [4]:
#ABILoading().get_abi_by_filename(abi_path)

In [5]:
reorg_mon = JSONRPCReorganizationMonitor(web3, check_depth=3)
reorg_mon.load_initial_block_headers(block_count=5)
processed_events = set()
latest_block = reorg_mon.get_last_block_live()
chain_reorg_resolution = reorg_mon.update_chain()
start, end = chain_reorg_resolution.get_read_range()

In [6]:
start = start - 10

In [7]:
rEvents = ReadEvents().apply(web3, start_block=start, end_block=end, filter=swap_filt)

In [8]:
events = {}
evt: LogResult
for k, evt in enumerate(rEvents):
    # How to uniquely identify EVM logs
    key = evt["blockHash"] + evt["transactionHash"] + evt["logIndex"]
    topics = evt["topics"]
    arguments = Conversion().decode_data(evt["data"])

    event = {}
    event['chain'] = chain
    event['contract'] = contract.lower()
    event['type'] = evt["event"].event_name.lower()
    event['platform'] = platform
    event['address'] = evt["address"]
    event['tx_hash'] = evt["transactionHash"]
    event['blk_num'] = evt["blockNumber"]
    event['timestamp'] = evt["timestamp"]
    event['details'] = {}
    event['details']['web3_type'] = evt["event"]
    event['details']['token0'] = Conversion().convert_uint256_hex_string_to_address(topics[1])
    event['details']['token1'] = Conversion().convert_uint256_hex_string_to_address(topics[2])
    event['details']['amount0In'] = Conversion().convert_int256_bytes_to_int(arguments[0]) 
    event['details']['amount1Out'] = Conversion().convert_int256_bytes_to_int(arguments[3])  
    
    events[k] = event
    if key not in processed_events:
        print(f"Swap at block:{evt['blockNumber']:,} tx:{evt['transactionHash']}")
        processed_events.add(key)
else:
    print(".")


Swap at block:60,958,014 tx:0xecd124aad29fb8043564bf9d1ec3bdfdf1e09201a05ca29ed0c19c41ea5cf0a4
Swap at block:60,958,014 tx:0x9c7e15803d4e64ca53916479756c457e872a10f32389fd2d553d46231c09d382
Swap at block:60,958,014 tx:0xe53cda9b996129bf2f15ed990bcd289ef0062dcb97e9804dd1d7d4a275229ba1
Swap at block:60,958,015 tx:0x920b63a20709b46a445d43e5ef5177a4103f184c094433b634683bf7424e9153
Swap at block:60,958,015 tx:0x7f73591d0137e01f8e357291a51f809d7dcbb1a4131cc22b1e3b407c57d159a1
Swap at block:60,958,016 tx:0x9fe60482bdf8d28748dbf57106bfc0a2148e2e4ead45be12970d79e2b39b05e7
Swap at block:60,958,016 tx:0x9fe60482bdf8d28748dbf57106bfc0a2148e2e4ead45be12970d79e2b39b05e7
Swap at block:60,958,016 tx:0x9fe60482bdf8d28748dbf57106bfc0a2148e2e4ead45be12970d79e2b39b05e7
Swap at block:60,958,017 tx:0x587a5589385d37a1b82bd04d619a006070395d5167d84ad12bfc06aba0658a86
Swap at block:60,958,017 tx:0xe7631299ab78452f6f1f3677338a79423234c8dfab82229ddaae663c4f6bf69c
Swap at block:60,958,017 tx:0x381658cd61a5b0271f2b

In [9]:
df_events = pd.DataFrame.from_dict(events, orient='index') 
df_events

,chain,contract,type,platform,address,tx_hash,blk_num,timestamp,details
0,polygon,uniswapv2pair,swap,sushi,0xa5cabfc725dfa129f618d527e93702d10412f039,0xecd124aad29fb8043564bf9d1ec3bdfdf1e09201a05c...,60958014,1724452007,{'web3_type': <class 'web3._utils.datatypes.Sw...
1,polygon,uniswapv2pair,swap,sushi,0xeefe518d89a49328580480fa3d90073bda074d32,0x9c7e15803d4e64ca53916479756c457e872a10f32389...,60958014,1724452007,{'web3_type': <class 'web3._utils.datatypes.Sw...
2,polygon,uniswapv2pair,swap,sushi,0x7162c0acf32820920a741d8fa466b8e6d60d530d,0xe53cda9b996129bf2f15ed990bcd289ef0062dcb97e9...,60958014,1724452007,{'web3_type': <class 'web3._utils.datatypes.Sw...
3,polygon,uniswapv2pair,swap,sushi,0xc4e595acdd7d12fec385e5da5d43160e8a0bac0e,0x920b63a20709b46a445d43e5ef5177a4103f184c0944...,60958015,1724452009,{'web3_type': <class 'web3._utils.datatypes.Sw...
4,polygon,uniswapv2pair,swap,sushi,0xa789324e64268c5385ea7678435fa83532705b0f,0x7f73591d0137e01f8e357291a51f809d7dcbb1a4131c...,60958015,1724452009,{'web3_type': <class 'web3._utils.datatypes.Sw...
5,polygon,uniswapv2pair,swap,sushi,0xeb6f7340b28ff2a3f66f335f94e3fc12a52b92d3,0x9fe60482bdf8d28748dbf57106bfc0a2148e2e4ead45...,60958016,1724452013,{'web3_type': <class 'web3._utils.datatypes.Sw...
6,polygon,uniswapv2pair,swap,sushi,0x8a73bc2a3382673b5572ab25369aa3cc6b024f6d,0x9fe60482bdf8d28748dbf57106bfc0a2148e2e4ead45...,60958016,1724452013,{'web3_type': <class 'web3._utils.datatypes.Sw...
7,polygon,uniswapv2pair,swap,sushi,0x6ff62bfb8c12109e8000935a6de54dad83a4f39f,0x9fe60482bdf8d28748dbf57106bfc0a2148e2e4ead45...,60958016,1724452013,{'web3_type': <class 'web3._utils.datatypes.Sw...
8,polygon,uniswapv2pair,swap,sushi,0x2813d43463c374a680f235c428fb1d7f08de0b69,0x587a5589385d37a1b82bd04d619a006070395d5167d8...,60958017,1724452015,{'web3_type': <class 'web3._utils.datatypes.Sw...
9,polygon,uniswapv2pair,swap,sushi,0xc4e595acdd7d12fec385e5da5d43160e8a0bac0e,0xe7631299ab78452f6f1f3677338a79423234c8dfab82...,60958017,1724452015,{'web3_type': <class 'web3._utils.datatypes.Sw...


In [10]:
events

{0: {'chain': 'polygon',
  'contract': 'uniswapv2pair',
  'type': 'swap',
  'platform': 'sushi',
  'address': '0xa5cabfc725dfa129f618d527e93702d10412f039',
  'tx_hash': '0xecd124aad29fb8043564bf9d1ec3bdfdf1e09201a05ca29ed0c19c41ea5cf0a4',
  'blk_num': 60958014,
  'timestamp': 1724452007,
  'details': {'web3_type': web3._utils.datatypes.Swap,
   'token0': '0x37af74b8096A6fd85bc4A36653a60B8D673BaEFc',
   'token1': '0x3cf7A3399439e3C10E2ea7e01272B54A66A04BA6',
   'amount0In': 14900077,
   'amount1Out': 895206}},
 1: {'chain': 'polygon',
  'contract': 'uniswapv2pair',
  'type': 'swap',
  'platform': 'sushi',
  'address': '0xeefe518d89a49328580480fa3d90073bda074d32',
  'tx_hash': '0x9c7e15803d4e64ca53916479756c457e872a10f32389fd2d553d46231c09d382',
  'blk_num': 60958014,
  'timestamp': 1724452007,
  'details': {'web3_type': web3._utils.datatypes.Swap,
   'token0': '0x411571B9c604b134c2CBA7f11aDff4fF14f004a3',
   'token1': '0xf678BCCeb109BC31eBe2cBb0b176d7b40e1f7054',
   'amount0In': 6509649